In [52]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
# from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt
from pyspark.mllib.evaluation import MulticlassMetrics
from sklearn.metrics import f1_score, recall_score

In [53]:
def init_spark():
    spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()
    return spark
spark_object = init_spark()

In [54]:
train_spark = spark_object.read.option("delimiter", ",").csv("./data/trainbalanced.csv", header='true', inferSchema='true')

In [56]:
for i in train_spark.columns:
    oldname = i
    if "." in oldname:
        newname = oldname.replace(".","")
        train_spark = train_spark.withColumnRenamed(oldname, newname)

In [57]:
train_data, test_data = train_spark.randomSplit([0.7, 0.3], seed=123)

In [58]:
input_list =[]
for i in train_spark.columns:
    if i != 'isFraud':
        input_list.append(i)

In [59]:
assembler = VectorAssembler(inputCols=input_list,outputCol='features')

In [60]:
train_features = assembler.transform(train_data)
test_features = assembler.transform(test_data)

In [61]:
lr = LogisticRegression(featuresCol='features', labelCol='isFraud', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_features)

In [62]:
lr_predictions = lr_model.transform(test_features)
# evaluator = RegressionEvaluator(labelCol='isFraud', predictionCol='prediction', metricName='rmse')
# rmse = evaluator.evaluate(lr_predictions)
# print('Root Mean Squared Error (RMSE) =', rmse)

In [63]:
y_pred=lr_predictions.select("prediction").collect()
y_orig=lr_predictions.select("isFraud").collect()

In [66]:
print(f1_score(y_orig, y_pred))
print(recall_score(y_orig, y_pred))

0.6663805508570851
1.0


In [ ]:
# evaluator = RegressionEvaluator(labelCol='isFraud', predictionCol='prediction', metricName='f1')
# rmse = evaluator.evaluate(lr_predictions)
# print('Root Mean Squared Error (RMSE) =', rmse)

In [ ]:
predictionAndLabels = lr_predictions.select("prediction","isFraud").rdd
multi_metrics = MulticlassMetrics(predictionAndLabels)

In [ ]:
f1_score = multi_metrics.fMeasure()

In [ ]:
pr = trainingSummary.pr.toPandas()
plt.plot(pr['recall'],pr['precision'])
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.show()